In [2]:
import numpy as np
import sqlite3
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from stop_words import get_stop_words
import shap
import warnings

warnings.filterwarnings("ignore")

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [3]:
romanian_texts = {}
moldavian_texts = {}

conn = sqlite3.connect('news.db')
c = conn.cursor()

c.execute('SELECT * FROM romania')
rows = c.fetchall()
for row in rows:
    if row[4] not in romanian_texts:
        romanian_texts[row[4]] = []
    romanian_texts[row[4]].append(row[5])
    
c.execute('SELECT * FROM moldova WHERE newspaper != "zugo"')
rows = c.fetchall()
for row in rows:
    text = ''
    if len(row[5]) > 10000:
        text = row[5][:10000]
    else:
        text = row[5]
    if row[4] not in moldavian_texts:
        moldavian_texts[row[4]] = []
        
    moldavian_texts[row[4]].append(text)

conn.close()

In [4]:
all_texts = {"romana": [], "moldova": []}

for key in romanian_texts:
    all_texts["romana"].extend(romanian_texts[key])

    
for key in moldavian_texts:
    all_texts["moldova"].extend(moldavian_texts[key])
    

In [4]:
!python -m spacy download ro_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 MB 3.5 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('ro_core_news_md')


In [4]:
# Extract most common parts of speech with spacy
import spacy

nlp = spacy.load('ro_core_news_md')

def get_most_common_pos(texts, n=10):
    pos = {}
    tokenized_texts = []
    for text in texts:
        doc = nlp(text)
        for token in doc:
            if token.pos_ not in pos:
                pos[token.pos_] = 0
            pos[token.pos_] += 1
            
        tokenized_texts.append([token.pos_ for token in doc])
        
    return [sorted(pos.items(), key=lambda x: x[1], reverse=True)[:n], tokenized_texts]

mc_romana, tk_romana = get_most_common_pos(all_texts["romana"])
mc_moldova, tk_moldova = get_most_common_pos(all_texts["moldova"])

In [18]:
from sklearn.linear_model import LogisticRegression

X = tk_romana + tk_moldova
y = ["romana"] * len(tk_romana) + ["moldova"] * len(tk_moldova)

X = np.array([' '.join(tag) for tag in X])
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

from sklearn.linear_model import LogisticRegression

sss = StratifiedShuffleSplit(n_splits=3, test_size=0.1, random_state=11)
text_clf = Pipeline(steps=[
        ('tfidf', TfidfVectorizer(min_df=3, max_df=0.7, max_features=10000, stop_words=get_stop_words('ro'))),
        ('clf', LogisticRegression(penalty='l2'))
    ], verbose=True)
parameters = {
    'tfidf__ngram_range': [(1, 4)],
    'tfidf__use_idf': (True, False),
    'clf__C': (0.1, 1, 10),
}
gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1, verbose=1)

scores = []
gs_scores = []

for train_index, test_index in sss.split(X_train, y_train):
    X_train_split, X_test_split = X_train[train_index], X_train[test_index]
    y_train_split, y_test_split = y_train[train_index], y_train[test_index]
    gs_clf = gs_clf.fit(X_train_split, y_train_split)
    
    scores.append(gs_clf.score(X_test_split, y_test_split))
    gs_scores.append(gs_clf.score(X_test, y_test))

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[Pipeline] ............. (step 1 of 2) Processing tfidf, total=  12.4s
[Pipeline] ............. (step 1 of 2) Processing tfidf, total=  12.1s
[Pipeline] ............... (step 2 of 2) Processing clf, total=   0.2s
[Pipeline] ............... (step 2 of 2) Processing clf, total=   0.2s
[Pipeline] ............. (step 1 of 2) Processing tfidf, total=  12.9s
[Pipeline] ............. (step 1 of 2) Processing tfidf, total=  12.8s
[Pipeline] ............. (step 1 of 2) Processing tfidf, total=  13.0s
[Pipeline] ............... (step 2 of 2) Processing clf, total=   0.1s
[Pipeline] ............... (step 2 of 2) Processing clf, total=   0.2s
[Pipeline] ............... (step 2 of 2) Processing clf, total=   0.2s
[Pipeline] ............. (step 1 of 2) Processing tfidf, total=  12.3s
[Pipeline] ............. (step 1 of 2) Processing tfidf, total=  13.5s
[Pipeline] ............... (step 2 of 2) Processing clf, total=   0.2s
[Pipeline] ......

In [19]:
print("Mean score: ", np.mean(scores))
print("Mean grid search score: ", np.mean(gs_scores))
print("Best parameters: ", gs_clf.best_params_)
print("Best score: ", gs_clf.best_score_)
print("Classification report:\n", classification_report(y_test, gs_clf.predict(X_test), ))

Mean score:  0.9980039920159681
Mean grid search score:  0.9920063948840928
Best parameters:  {'clf__C': 10, 'tfidf__ngram_range': (1, 4), 'tfidf__use_idf': False}
Best score:  0.9970005546311702
Classification report:
               precision    recall  f1-score   support

     moldova       0.99      1.00      0.99       631
      romana       1.00      0.96      0.98       203

    accuracy                           0.99       834
   macro avg       0.99      0.98      0.99       834
weighted avg       0.99      0.99      0.99       834



In [5]:
from cube.api import Cube

cube_nlp = Cube(verbose=True)

cube_nlp.load("ro")



Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a m

In [6]:
def get_pos_from_texts(texts):
    texts_upos = []
    texts_xpos = []
    for text in texts:
        tmic_upos = []
        tmic_xpos = []
        while len(text) > 512:
            try:
                small_text = ''
                text = text.split(' ')
                while len(small_text) + len(text[0]) <= 512:
                    small_text += text[0]
                    text = text[1:]
                text = ' '.join(text)
                doc = None
                doc = cube_nlp(small_text)
                
                text_upos = []
                text_xpos = []
                for sent in doc:
                    for word in sent.words:
                        text_upos.append(word.upos)
                        text_xpos.append(word.xpos)
                
                # print(1)
                tmic_upos.append([str(token) for token in text_upos])
                tmic_xpos.append([str(token) for token in text_xpos])
            except:
                # print(3)
                # print(small_text)
                # print(text)
                continue
            
        try:
            doc = None
            doc = cube_nlp(text).sentences
        except:
            # print(4)
            continue
        
        # print(len(doc))
        # print(type(doc[0]))
        
        # break
        
        text_upos = []
        text_xpos = []
        for sent in doc:
            for word in sent.words:
                text_upos.append(word.upos)
                text_xpos.append(word.xpos)
        
        # print(2)
        tmic_upos.append([str(token) for token in text_upos])
        tmic_xpos.append([str(token) for token in text_xpos])
        
        tmediu_upos = []
        tmediu_xpos = []
        
        for i in range(len(tmic_upos)):
            tmediu_upos.extend(tmic_upos[i])
            tmediu_xpos.extend(tmic_xpos[i])
        
        texts_upos.append(tmediu_upos)
        texts_xpos.append(tmediu_xpos)
        
        print(len(texts_upos))
        
        
    return [texts_upos, texts_xpos]

print("ro-cube")
print(len(all_texts["romana"]))
upos_romana, xpos_romana = get_pos_from_texts(all_texts["romana"])

print("md-cube")
print(len(all_texts["moldova"]))
upos_moldova, xpos_moldova = get_pos_from_texts(all_texts["moldova"])

ro-cube
1018
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274

In [ ]:
# Train a classifier using the most common parts of speech
X = upos_romana + upos_moldova
y = ["romana"] * len(upos_romana) + ["moldova"] * len(upos_moldova)

X = np.array([' '.join(tag) for tag in X])
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

sss = StratifiedShuffleSplit(n_splits=5, test_size=0.1, random_state=11)
text_clf = Pipeline(steps=[
        ('tfidf', TfidfVectorizer(min_df=3, max_df=0.7, max_features=10000)),
        ('clf', LogisticRegression(penalty='l2'))
    ], verbose=True)
parameters = {
    'tfidf__ngram_range': [(1, 4)],
    'tfidf__use_idf': (True, False),
    'clf__C': (0.1, 1, 10),
}
gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1, verbose=1)

scores = []
gs_scores = []

for train_index, test_index in sss.split(X_train, y_train):
    X_train_split, X_test_split = X_train[train_index], X_train[test_index]
    y_train_split, y_test_split = y_train[train_index], y_train[test_index]
    gs_clf = gs_clf.fit(X_train_split, y_train_split)
    
    scores.append(gs_clf.score(X_test_split, y_test_split))
    gs_scores.append(gs_clf.score(X_test, y_test))

In [ ]:
print("Mean score: ", np.mean(scores))
print("Mean grid search score: ", np.mean(gs_scores))
print("Best parameters: ", gs_clf.best_params_)
print("Best score: ", gs_clf.best_score_)
print("Classification report: ", classification_report(y_test, gs_clf.predict(X_test), ))

In [ ]:
from sklearn.linear_model import LogisticRegression

X = tk_romana + tk_moldova
y = ["romana"] * len(tk_romana) + ["moldova"] * len(tk_moldova)

X = np.array([' '.join(tag) for tag in X])
y = np.array(y)

sss = StratifiedShuffleSplit(n_splits=5, test_size=0.1, random_state=11)
text_clf = Pipeline(steps=[
        ('tfidf', TfidfVectorizer(min_df=3, max_df=0.7, max_features=10000, stop_words=get_stop_words('ro'))),
        ('clf', LogisticRegression(penalty='l2'))
    ], verbose=True)
parameters = {
    'tfidf__ngram_range': [(1, 4)],
    'tfidf__use_idf': (True, False),
    'clf__C': (0.1, 1, 10),
}
gs_clf = GridSearchCV(text_clf, parameters, cv=sss, n_jobs=-1, verbose=1)

scores = []
gs_scores = []

for train_index, test_index in sss.split(X_train, y_train):
    X_train_split, X_test_split = X_train[train_index], X_train[test_index]
    y_train_split, y_test_split = y_train[train_index], y_train[test_index]
    gs_clf = gs_clf.fit(X_train_split, y_train_split)
    
    scores.append(gs_clf.score(X_test_split, y_test_split))
    gs_scores.append(gs_clf.score(X_test, y_test))

In [ ]:
print("Mean score: ", np.mean(scores))
print("Mean grid search score: ", np.mean(gs_scores))
print("Best parameters: ", gs_clf.best_params_)
print("Best score: ", gs_clf.best_score_)
print("Classification report: ", classification_report(y_test, gs_clf.predict(X_test), ))